In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
import pandas as pd
from datasets import load_dataset, DatasetDict, load_dataset
from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
import fasttext
import csv

In [ ]:
model = fasttext.load_model("./indicnlp.ft.ta.300.bin")

In [ ]:
def convert_to_fasttext_format(filename):
    df = pd.read_csv(f"./dataset/{filename}.csv")
    df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: '__label__' + str(x))
    df[["label", "text"]].to_csv(f'./dataset/{filename}.txt', 
      index = False, 
      sep = ' ',
      header = None, 
      quoting = csv.QUOTE_NONE, 
      quotechar = "", 
      escapechar = " ")

In [ ]:
# convert_to_fasttext_format("train")
# convert_to_fasttext_format("validation")
# convert_to_fasttext_format("test")
convert_to_fasttext_format("resampled_train")

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("dataset/train.csv")

In [ ]:
train = pd.read_csv("dataset/train.csv")
oversampler = RandomOverSampler(sampling_strategy=0.5)
text, label = oversampler.fit_resample(train["text"].to_numpy().reshape(-1,1), train["label"])
resampled_train = pd.DataFrame({
    "text": text.flatten(),
    "label": label
})
resampled_train.to_csv("./dataset/resampled_train.csv", index=False)

In [ ]:
model = fasttext.train_supervised('dataset/train.txt', wordNgrams = 2, epoch=500)

In [ ]:
model.test("dataset/test.txt", k=1)

In [ ]:
model.predict("என்ன  இழவுடா  இது  கங்கனாக்கு  தமிழே  தெரியாது  அவங்களை  வைத்து  ஜெயலலிதா  அவர்களின்  வாழ்க்கை  கதை  எடுக்க  உங்களுக்கு  எப்படி  மனம்  வந்தது  விஜய்  அவர்களே")[0][0]

In [ ]:
test = pd.read_csv("./dataset/test.csv")

In [ ]:
words = ["கூமுட்டை",
"கம்மனாட்டி",
"சாவுகிராக்கி",
"பொறம்போக்கு",
"கிழட்டு",
"புண்டை",
"கருமம்",
"கருமான்றம்",
"போட்ட",
"ஓத்தா",
"மயிறு",
"கூதி",
"சுன்னி",
"சூத்து",
"சூத்து",
"தாயொலி",
"தாயோளி",
"தங்கையொலி",
"தங்கையொளி",
"தேவுடியா",
"பூல்",
"பூளூம்பி",
"ஊம்பு",
"லவதே",
"கருவாய்",
"கெழட்டுகூதி",
"கேனக்கூதி",
"கேன",
"முண்டம்",
"சாபக்கேடு",
"கிழட்டு",
"கொட்ட",
"புடுங்கி",
"குண்டி",
"மாமாபயலே",
"உங்கொப்பன்",
"பொட்டக்கூதி",
"போட்டபாடு",
"பிண்டகலப்பை",
"சப்பு",
"புண்டாமவன்",
"கும்மாள",
"சூதெறிச்சல்",]

In [ ]:
from sentence_transformers import SentenceTransformer, util
import tqdm

In [ ]:
transformer = SentenceTransformer('ai4bharat/indic-bert')

In [ ]:
screen_word_embeddings = []
for word in words:
    screen_word_embeddings.append(transformer.encode(word, convert_to_tensor=True))

In [ ]:
preds = []
preds_ = []
for text in tqdm.tqdm(test["text"]):
    model_prediction = int(model.predict(text)[0][0].split("__")[-1])
    if any(word in text.split() for word in words):
        preds.append(1)
    else:
        preds.append(model_prediction)
    preds_.append(model_prediction)

In [ ]:
labels = []
for label in test["label"]:
    labels.append(label)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [ ]:
accuracy_score(labels, preds)

In [ ]:
f1_score(preds, labels, average="weighted")

In [ ]:
accuracy_score(labels, preds_)

In [ ]:
f1_score(preds_, labels, average="weighted")

In [ ]:
confusion_matrix(labels, preds_)

In [ ]:
confusion_matrix(labels, preds)

In [ ]:
import numpy as np

In [ ]:
failed_values = test["text"].to_numpy()[np.where((np.array(preds) != np.array(preds_)))]

In [ ]:
"கேன" in "நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு எப்படா திரௌபதி ரிலீஸ் ஆகுமின்னு எதிர்பாத்திட்ரிக்கேன்"

In [ ]:
for text in test["text"]:
    if "ஓத்தா" in text.split():
        print(text)

In [ ]:
from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')